# Reinceforcement learning -scheduling using simpy with dqn

### import 

In [2]:
import simpy
import pandas as pd
import numpy as np 
import random 
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from collections import deque 
from itertools import product
import math
#from DQN import DQN
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam 
import time

### dqn

3개의 source, destnation에 대한 min rate array를 state로 한다
min rate란 source에서 보내야 하는 filesize(단위?) / deadline까지 남은 시간 (초?) 
 -> mdp는 state에 대한 가치함수를 얻기 때문에 state가 간단해야 함

In [7]:
class DQN:#모델 선언
    def __init__(self): #parameter들의 초기값
        #self.gamma = 0.85
        self.epsilon = 0.999
        self.epsilon_min = 0.01
        #self.epsilon_decay = 0.95
        self.step = 1
        self.tau = 0.125 #?
        self.learning_rate = 1
        self.memory = deque()
        self.model = self.create_model() #현재 state에 대한 model
        self.target_model = self.create_model() #next state에 대한 model

    # create the neural network to train the q function 
    def create_model(self): #Q값예측모델. 
        model = Sequential()
        model.add(Dense(24, input_dim= 3, activation= 'relu')) # input dimension : source들 차원
        model.add(Dense(48, activation= 'relu'))
        model.add(Dense(24, activation= 'relu'))
        model.add(Dense(66)) #계산했을 때,(1~10)까지 세 수의 합이 10이 되는 경우의수는 66개. output에 대한 가중치는 매번 update되기 때문에 이에 mapping시키면 된다. 
        model.compile(loss= 'mean_squared_error', optimizer= Adam(lr= self.learning_rate))
        
        return model 



    # Action function to choose the best action given the q-function if not exploring based on epsilon p값에 의한 예측이 아닐때
    def choose_action(self, state, allowed_actions): #action을 선택 (parameter로 선택가능한 action이 들어옴)
        select = False
        if (self.step%10000 == 0):#약 10만번 step에서 10000번마다 epsilon이 감소
            self.epsilon = max(self.epsilon_min, pow(self.epsilon,int(self.step/10000 +1)))
        print ("epsilon", self.epsilon)
        self.step+=1
        r = np.random.random()
        if r < self.epsilon: #p값보다 작은 경우 랜덤한 액션을 취함
            print("random action")
            return random.choice(allowed_actions),self.step,select
        
        
        print ("@@action choose@@" , self.step)
        select = True
        state = np.array(state).reshape(1,len(state)) #p값보다 큰경우, state 배열 생성
        
        pred = self.model.predict(state)[0]
        #print ("q",pred)
        
        return self.maxQ_action(pred,allowed_actions),self.step,select #Q예측값중 min_rate 이상으로 가장 큰 action을 선택
    

    def maxQ_action(self,pred,allowed_actions):#allowed action 생성 (min_rate 이상 조합만 남김)
        print ("max q", np.argmax(pred) )
        return allowed_actions[np.argmax(pred)]
        
        
        
    # create replay buffer memory to sample randomly #메모리에서 꺼내서 학습할 수 있게 저장, terminal이란 next_state가 없는 경우
    def remember(self, state, action, reward, next_state,terminal):
        self.memory.append([state, action, reward, next_state,terminal])


    # build the replay buffer 저장한 것을 버퍼에서 꺼내오는.? 학습단계?
    def replay(self,allowed_actions):
        
        #global mse_loss
        mse=[]
        batch_size = 32
        if len(self.memory) < batch_size: #buffer에 저장된 memory가 buffer의 총 batch_size보다 작다면 return
            return 
        
        samples = random.sample(self.memory, batch_size) #메모리에서 배치사이즈만큼 랜덤으로 선택
        
        
        for sample in samples:
            
            #print ("sample" , sample)
            
            state, action, reward, new_state, terminal = sample # sample 데이터 하나를 꺼내서
            
            state = np.array(state).reshape(1,len(state)) 
            
            new_state = np.array(new_state).reshape(1,len(new_state))
            
            target = self.target_model.predict(state) 
            
            action_id = allowed_actions.index(tuple(action)) #63개의 allowed action 중에서 state에 대한 action의 index를 추출

            if terminal :
                target[0][action_id] = reward
            else :
                next_pred = self.target_model.predict(new_state)[0] #new state에 대한 target 예측
                  
                Q_future= max(next_pred) #next state에 대한 predict 값 중 가장 큰 값이 Q값이 됨
                
                target[0][action_id] = reward + Q_future * self.learning_rate # target의 action_id번째 위치에 다음 Q값이 들어감. 맞춰야 하는 값!!!
            
            history=self.model.fit(state, target, epochs= 1, verbose= 0) 
            mse.append(history.history['loss'][0]) # loss 기록
            
        return min(mse)
        
        #print("Mean_square_error:"min(mse_loss))
        


    # update our target network 
    def train_target(self): #target network를 업데이트
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)#loss함수?
            #target_weights[i] = weights[i]
        self.target_model.set_weights(target_weights)



    # save our model 
    def save_model(self, fn):
        self.model.save(fn)



model은 현재 state에 대한 q값을 예측

target_model은 action에 의한 다음 state'에서의 q' 값을 예측

현재 q값이란 reward + q' 가 되므로,

model은 reward + q' 를 target으로 하는데

그렇기 때문에 target_model이 맞춰야 하는 q' 또한 정확하게 학습이 되어야 함

train(replay) 과정에서는 model을 state를 넣으면 올바른 target (reware + q')를 산출하도록 학습시키고,

weight update 과정에서는 train_model에 학습된 model의 weight를 넣어준다.

train_model도 결국은 model과 같은 매커니즘으로 알맞은 q 예측값을 산출해야하기 때문임


=> model의 predict값을 확인한 결과, model weight update에는 문제가 없었고 좋지 않은 state로 흘러감에도 불구하고 pred값이 전혀 감소하지 않음 즉, reward function의 문제

=> reward function은 현재 "Active flow"에 대해 책정하는 것이 핵심이다. 만약 가장 큰 deadline이 2초 이고, deadline이 완료된 flow의 경우 action을 많이 할당할 수록 reward값이 커지기 때문에 결국 만기된 플로우에 더 큰 pacing rate을 할당하도록 설계된다.

In [ ]:
#보상 함수 : deactive flow가 누적되서 보상을 받는 것을 방지      
def reward_function(deadlines,action,value): #value None이면 active flow
    #deactive flow를 제외, deadline이 0인 것은 제외
    reward = 0
    drem_max=np.max(deadlines) #최대 deadline
    for dremi in range(len(deadlines)):
        #print ("drem",deadlines[dremi])
        #print ("value",value[dremi])
        if ((value[dremi] != 1) and (value[dremi] != 0) and (deadlines[dremi] > 0)) : #active flow, 전송중
            reward += ((drem_max - deadlines[dremi])*action[dremi])
        elif ((value[dremi] != 1) and (value[dremi] != 0)  and (deadlines[dremi] ==0)) : #active flow, deadline 지남
            reward -=1
    #print ("reward",reward)
    return reward

    
def episode(env,DQN,Tsc,Tfu,allowed_actions): #pacing rate가 각 flow에게 할당
    global action
    global state
    global request
    global flow_success
    global record
    
    cnt=1 #episode 수
    c=0 #scheduling interval의 수
   
    #episode 시작
    
    while True: #Simularion time 동안 episode를 반복한다
        print ("--------------------------------------------------")
        print("********Episode start********",cnt)
        print ("")
        
        first_action=1 #new action policy -> EDF
        
        #1개의 에피소드는 모든 filesize가 0이 될때까지 실행
        while ((request['filesize'][0]!=0)or(request['filesize'][1]!=0)or(request['filesize'][2]!=0)):#모두 전송이 완료될 때 까지
            c+=1
            
            #state 결정
            
            state=[0,0,0] #state 초기화
            for s in range(len(sources)):#deadline이 0이 아닌 source는 그대로
                if (request['value'][s] == 1) : #deactive (deadline =0 , value =1)
                    state[s]=0
                else: #active, value=none
                    if (request['deadline'][s] > 0) : #active (value = None, deadline >0이면 전송중, deadline =0이면 기한 지남)
                        state[s]=math.ceil(request['filesize'][s]/request['deadline'][s])
                    elif (request['deadline'][s] == 0):# 기한 지남
                        state[s]=10 #link capacity 전체 할당
            
            print ("state", state)

            #action 결정
            
            if (first_action==1): #New state-action policy를 EDF방식으로 (가장 deadline이 시급한)     
                action=[0,0,0]
                index=np.argmin(request['deadline']) #deadline 최소인 source의 index
                action[index]=10 #bottleneck capacity
                
            else: 
                action,p,select=dqn_agent.choose_action(state,allowed_actions) #DQN에 의한 액션선택
                pexp.append(p) #epsilon값
                
            print ("action", action)
                
            #Scheduling interval 시작
            print ("Tsc" , c)
            
            for i in range(Tsc): 
                for s in range(len(sources)): #각 source에 대해 
                    
                    #filesize와 deadline 감소
                    
                    request['filesize'][s]=max([request['filesize'][s]-int(action[s]),0]) #filesize는 음수 X
                    
                    if (request['value'][s]==None):
                        request['deadline'][s]= request['deadline'][s]-1 #deadline는 음수 가능, active flow에대해서만 감소
                    
                    # Active, Deactive flow 검사
                    
                    if ((request['filesize'][s]==0) and (request['value'][s]==None)): #아직 완료되지 않았던 flow가 전송이 완료되면?
                        
                        if (request['deadline'][s]>=0): #기간 안에 전송되면? 남아있는 시간이 양수, 또는 0 (시간이 0에 딱 맞게 전송 되는 경우도 있음..)
                            request['value'][s]=1 #value를 1로 변경
                            flow_success.append(1)
                            #print ("s{}의 전송이 deadline 안에 완료됨".format(s))
                            
                        else: #기간안에 전송된게 아니라면(value초기값은 None)
                            request['value'][s]=0
                            flow_success.append(0)
                            #print ("s{}의 전송이 deadline을 지나 완료됨".format(s))
                            
                yield env.timeout(Tfu)# Tfu(1초)마다 위 과정 실행
            
            
            
            #모든 전송이 완료된 후 next_state는 고려할 필요 없음: terminal=True로 하여 target에 reward를 할당
            
            if ((request['filesize'][0]==0)and(request['filesize'][1]==0)and(request['filesize'][2]==0)):
                terminal = True
            else : 
                terminal = False
            
            #Next state 결정
            
            next_state=[0,0,0]
            for s in range(len(sources)):
                if (request['value'][s] == 1) : 
                    next_state[s]=0
                else: #active, value=none
                    if (request['deadline'][s] > 0) : 
                        next_state[s]=math.ceil(request['filesize'][s]/request['deadline'][s])
                    elif (request['deadline'][s] == 0):
                        next_state[s]=10 #link capacity

            #print("next_state" , next_state)
                        
            reward = reward_function((request['deadline']),action,request['value'])
            cur_state = state 
            action = action
            new_state = next_state 
            reward = reward
            terminal = terminal
            
            if (first_action==0): # 첫번쨰 선택 액션이 아닌경우에만 학습
                dqn_agent.remember(cur_state, action, reward, new_state,terminal) #새로운 state로 설정해주고 기존state저장
                mse_loss.append(dqn_agent.replay(allowed_actions))#학습, loss 저장
                dqn_agent.train_target()
                record.append([cur_state, action, reward, new_state , select]) #select는 action을 random에 의해 선택했는지 dqn에 의해 선택했는지 여부
            
            first_action=0 

        for i in range(len(sources)):
            if (request['value'][i]==1):
                print ("source {} 전송완료".format(i))
            else:
                print ("source {} deadine 충족하지 못함".format(i))
        
        print("모두 전송 완료")
        
        #다음 episode에 simulation할 flow생성

        filesize=[random.randrange(10,50)for source in range(len(sources))] 
        deadline=[int(filesize[source]/3) for source in range(len(sources))] #sum_rmin이  9
        request = {
            'sources' : sources,
            'destinations' : destinations, 
            'filesize' : filesize, #단위는 Gbps
            'deadline' : deadline,
            'value' : [None for source in sources] #아직 전송되지 않았으면 None, 제시간에 전송되었으면 1, 제시간에 전송되지 않았으면 0
        }

        state=[0,0,0]#초기 state
        action=[0,0,0]#초기 action ->고칠것
        print ("********episode end********")
        print("--------------------------------------------------")
        cnt+=1
    
    
    
# main함수

env = simpy.Environment()
dqn_agent = DQN()

#npz
mse_loss=[] #episode의 진행에 따른 mse_loss의 변화율 graph
flow_success=[] #episode의 진행에 따른 flow_success rate
pexp=[] #action choice 진행에 따른 pexp 변화
record = []

state=[0,0,0]#초기 state
action=[0,0,0]#초기 action ->고칠것
Tsc=3 #scheduling interval을 구성하는 flow update interval의 수
Tfu=1 #flow update interval의 시간단위(0.1초로 가정)
sources = ['s0', 's1', 's2']
destinations = ['d0', 'd1', 'd2']
filesize=[random.randrange(10,50) for source in range(len(sources))] #단위 Gb
deadline=[int(filesize[source]/3) for source in range(len(sources))] #sum_rmin이  9Gbps, 각각 3
request = {
    'sources' : sources,
    'destinations' : destinations, 
    'filesize' : filesize, #단위는 Mbps
    'deadline' : deadline,
    'value' : [None for source in sources] #아직 전송되지 않았으면 None, 제시간에 전송되었으면 1, 제시간에 전송되지 않았으면 0
}

A=list(range(11))
B=list(range(11))
C=list(range(11)) 
allowed_actions=[] #합이 10이 되는 0~10까지의 수 조합
for i in list(product(*(A,B,C))):
    if sum(list(i))==10:
        allowed_actions.append(i) #d는 66개의 조합
        
start = time.time()  # 시작 시간 저장
env.process(episode(env,DQN,Tsc,Tfu,allowed_actions))
env.run(until=100000)#10만 초 동안 가동

#결과 저장
np.savez('simulation history 1029_notebook',loss = mse_loss, success = flow_success, p = pexp, record = record )
dqn_agent.save_model("dqn_policy.h5")

print("종료")
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간


--------------------------------------------------
********Episode start******** 1

state [4, 4, 3]
action [10, 0, 0]
Tsc 1
state [0, 4, 5]
epsilon 0.999
random action
action (1, 8, 1)
Tsc 2
state [0, 3, 11]
epsilon 0.999
random action
action (0, 3, 7)
Tsc 3
state [0, 3, 0]
epsilon 0.999
random action
action (1, 4, 5)
Tsc 4
state [0, 1, 0]
epsilon 0.999
random action
action (4, 0, 6)
Tsc 5
state [0, 4, 0]
epsilon 0.999
random action
action (0, 6, 4)
Tsc 6
source 0 전송완료
source 1 전송완료
source 2 deadine 충족하지 못함
모두 전송 완료
********episode end********
--------------------------------------------------
--------------------------------------------------
********Episode start******** 2

state [4, 4, 3]
action [0, 10, 0]
Tsc 7
state [5, 0, 5]
epsilon 0.999
random action
action (1, 2, 7)
Tsc 8
state [6, 0, 2]
epsilon 0.999
random action
action (0, 10, 0)
Tsc 9
state [12, 0, 10]
epsilon 0.999
random action
action (0, 2, 8)
Tsc 10
state [10, 0, 0]
epsilon 0.999
random action
action (0, 1, 9)
Tsc 11
s

state [0, 5, 0]
epsilon 0.999
random action
action (9, 0, 1)
Tsc 84
state [0, 9, 0]
epsilon 0.999
random action
action (4, 1, 5)
Tsc 85
state [0, 10, 0]
epsilon 0.999
random action
action (0, 1, 9)
Tsc 86
state [0, 0, 0]
epsilon 0.999
random action
action (0, 0, 10)
Tsc 87
state [0, 0, 0]
epsilon 0.999
random action
action (8, 0, 2)
Tsc 88
state [0, 0, 0]
epsilon 0.999
random action
action (3, 5, 2)
Tsc 89
state [0, 0, 0]
epsilon 0.999
random action
action (2, 0, 8)
Tsc 90
state [0, 0, 0]
epsilon 0.999
random action
action (1, 0, 9)
Tsc 91
state [0, 0, 0]
epsilon 0.999
random action
action (1, 3, 6)
Tsc 92
source 0 deadine 충족하지 못함
source 1 deadine 충족하지 못함
source 2 전송완료
모두 전송 완료
********episode end********
--------------------------------------------------
--------------------------------------------------
********Episode start******** 15

state [4, 4, 3]
action [10, 0, 0]
Tsc 93
state [0, 9, 4]
epsilon 0.999
random action
action (1, 5, 4)
Tsc 94
state [0, 0, 4]
epsilon 0.999
random act

source 0 전송완료
source 1 deadine 충족하지 못함
source 2 deadine 충족하지 못함
모두 전송 완료
********episode end********
--------------------------------------------------
--------------------------------------------------
********Episode start******** 27

state [4, 4, 4]
action [10, 0, 0]
Tsc 167
state [1, 5, 5]
epsilon 0.999
random action
action (8, 0, 2)
Tsc 168
state [0, 7, 7]
epsilon 0.999
random action
action (5, 2, 3)
Tsc 169
state [0, 11, 17]
epsilon 0.999
random action
action (3, 0, 7)
Tsc 170
state [0, 10, 0]
epsilon 0.999
random action
action (4, 4, 2)
Tsc 171
state [0, 0, 0]
epsilon 0.999
random action
action (5, 3, 2)
Tsc 172
state [0, 0, 0]
epsilon 0.999
random action
action (2, 4, 4)
Tsc 173
source 0 전송완료
source 1 deadine 충족하지 못함
source 2 deadine 충족하지 못함
모두 전송 완료
********episode end********
--------------------------------------------------
--------------------------------------------------
********Episode start******** 28

state [4, 3, 4]
action [0, 10, 0]
Tsc 174
state [4, 1, 4]
epsilon 0

state [0, 0, 10]
epsilon 0.999
random action
action (8, 2, 0)
Tsc 245
state [0, 0, 10]
epsilon 0.999
random action
action (4, 2, 4)
Tsc 246
state [0, 0, 0]
epsilon 0.999
random action
action (5, 0, 5)
Tsc 247
state [0, 0, 0]
epsilon 0.999
random action
action (3, 0, 7)
Tsc 248
source 0 전송완료
source 1 전송완료
source 2 deadine 충족하지 못함
모두 전송 완료
********episode end********
--------------------------------------------------
--------------------------------------------------
********Episode start******** 42

state [3, 3, 4]
action [0, 10, 0]
Tsc 249
state [5, 0, 5]
epsilon 0.999
random action
action (3, 7, 0)
Tsc 250
state [5, 0, 13]
epsilon 0.999
random action
action (4, 3, 3)
Tsc 251
state [6, 0, 0]
epsilon 0.999
random action
action (3, 6, 1)
Tsc 252
state [0, 0, 0]
epsilon 0.999
random action
action (9, 1, 0)
Tsc 253
state [0, 0, 0]
epsilon 0.999
random action
action (6, 3, 1)
Tsc 254
state [0, 0, 0]
epsilon 0.999
random action
action (4, 5, 1)
Tsc 255
state [0, 0, 0]
epsilon 0.999
random ac

source 0 deadine 충족하지 못함
source 1 전송완료
source 2 전송완료
모두 전송 완료
********episode end********
--------------------------------------------------
--------------------------------------------------
********Episode start******** 54

state [3, 4, 3]
action [10, 0, 0]
Tsc 329
state [0, 7, 8]
epsilon 0.999
random action
action (4, 3, 3)
Tsc 330
state [0, 10, 0]
epsilon 0.999
random action
action (4, 1, 5)
Tsc 331
state [0, 0, 0]
epsilon 0.999
random action
action (5, 1, 4)
Tsc 332
state [0, 0, 0]
epsilon 0.999
random action
action (3, 0, 7)
Tsc 333
state [0, 0, 0]
epsilon 0.999
random action
action (7, 3, 0)
Tsc 334
source 0 전송완료
source 1 deadine 충족하지 못함
source 2 deadine 충족하지 못함
모두 전송 완료
********episode end********
--------------------------------------------------
--------------------------------------------------
********Episode start******** 55

state [4, 3, 4]
action [10, 0, 0]
Tsc 335
state [0, 4, 7]
epsilon 0.999
random action
action (5, 0, 5)
Tsc 336
state [0, 6, 10]
epsilon 0.999
random 

In [ ]:
memory

In [4]:
flow_success

[1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,


In [ ]:
len(flow_success)

In [6]:
rate=[]
flow_success=np.array(flow_success)
for i in range(int(len(flow_success)/15)):#200개당평균
    print(flow_success[15*i:15*(i+1)])
    rate.append(np.mean(flow_success[15*i:15*(i+1)]))
    print(np.mean(flow_success[15*i:15*(i+1)]))


[1 0 0 1 0 0 1 0 0 1 1 0 1 0 0]
0.4
[1 0 0 1 0 0 1 0 1 1 0 0 1 0 0]
0.4
[1 0 0 1 0 0 1 1 0 1 1 0 1 0 0]
0.4666666666666667
[1 1 0 1 0 0 1 1 0 1 1 0 1 0 0]
0.5333333333333333
[1 0 0 1 0 0 1 0 0 1 0 0 1 0 0]
0.3333333333333333
[1 1 0 1 0 0 1 1 0 1 1 0 1 0 0]
0.5333333333333333
[1 1 0 1 1 0 1 0 1 1 0 0 1 1 0]
0.6
[1 0 0 1 0 0 1 0 0 1 1 0 1 0 0]
0.4
[1 0 0 1 0 0 1 0 0 1 0 0 1 0 0]
0.3333333333333333
[1 1 0 1 0 0 1 0 0 1 0 0 1 0 0]
0.4
[1 0 0 1 0 0 1 0 0 1 1 0 1 1 0]
0.4666666666666667
[1 0 0 1 0 0 1 0 1 1 0 0 1 0 0]
0.4
[1 0 0 1 0 0 1 0 0 1 0 0 1 0 0]
0.3333333333333333
[1 0 1 1 0 0 1 1 0 1 0 0 1 1 0]
0.5333333333333333
[1 1 0 1 0 0 1 1 0 1 0 0 1 1 0]
0.5333333333333333
[1 0 0 1 0 0 1 0 0 1 1 0 1 0 0]
0.4
[1 0 0 1 0 0 1 1 0 1 0 0 1 0 0]
0.4
[1 0 0 1 0 0 1 0 0 1 1 0 1 0 0]
0.4
[1 1 0 1 0 0 1 0 0 1 1 0 1 0 0]
0.4666666666666667
[1 0 0 1 0 0 1 0 0 1 0 0 1 1 0]
0.4
[1 0 0 1 0 0 1 0 0 1 0 0 1 0 0]
0.3333333333333333
[1 1 0 1 1 0 1 1 0 1 1 0 1 0 0]
0.6
[1 0 0 1 0 0 1 0 0 1 0 0 1 0 0]
0.333333333

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

# x = mse_loss

# plt.plot(x)
# plt.show()

#flow_success_rate=
x2 = rate

plt.plot(x2)
plt.show()

In [ ]:
len(mse_loss)